
From the book (chapter 14.4):

>The US Department of Agriculture makes available a database of food nutrient information. Programmer Ashley Williams made available a version of this database in JSON format.

___



In [ ]:
library(tidyverse)
library(magrittr)
library(jsonlite)


___

## Loading the data

This data come from the US Department of Agriculture and report a set of food nutrient information. These data were made available by Ashley Williams in JSON format, where each food has a number of identifying attributes along with two lists of nutrients and portion sizes.



In [ ]:
path <- "../datasets/usda_food/database.json"
db <- as_tibble(fromJSON(path, flatten = T))

In [ ]:
db


Some of the features in this dataset are actually nested dataframes, as in `nutrients`.



In [ ]:
db[[1, "nutrients"]]


Without going too much in details, let's grep a subset of these information by selecting the `id`, `description`, `group` and `manufacturer` columns. In addition, we will rename the `description` and `group` columns to `food` and `fgroup`, respectively.



In [ ]:
info <- db %>%
    select(id, description, group, manufacturer) %>%
    rename(food = description, fgroup = group)


Now let's see the distribution of food groups.



In [ ]:
info %>% count(fgroup, sort = T)


Let's collect all of the nutrients into a single dataframe. First we will create a vector with the ids that will then be added to each nutrient dataframe.



In [ ]:
ids <- c()
for (elem in db$id) {
    ids <- append(ids, rep(elem, dim(subset(db, id == elem)[["nutrients"]][[1]])[1]))
}

In [ ]:
nutr_rows <- bind_rows(db$nutrients)

In [ ]:
nutrients <-
    bind_cols(id = ids, nutr_rows) %>%
    rename(nutrient = description, nutgroup = group)

In [ ]:
nutrients



Now we can merge the previously created `info` dataframe with `nutrients`.



In [ ]:
ndata <- full_join(info, nutrients, by = "id")

In [ ]:
ndata



Let's calculate median values by food group and nutrient type.



In [ ]:
result <- ndata %>%
    group_by(nutrient, fgroup) %>%
    summarise(quant = median(value))

In [ ]:
result



Now we can view which foods contain most Zinc.



In [ ]:
result %>%
    filter(nutrient == "Zinc, Zn") %>%
    ggplot(aes(x = reorder(fgroup, quant), y = quant, fill = fgroup)) +
    geom_col() +
    coord_flip() +
    guides(fill = F) +
    labs(x = "Food group", y = "Median value", title = "Median Zinc content")


___



In [ ]:
sessionInfo()